In [1]:
# PROJET 1:
# Notre programme permet de tracer une courbe en 3D, les lignes de niveaux, calculer le vecteur gradient et calculer la matrice Hessienne
# ainsi que l’implémentation des méthodes de gradient à pas fixe, variable et optimal d\'une fonction dans R2")
# realiser par : Skander AMDOUNI / Ala MABROUK/ EskanderBELHAJ AMOR	/ Nerimen MECHERGUI / DoniezTOUIL / Omar ACHOUR


In [2]:
#Implémentation des bibliothéques nécessaires : 
from sys import exit
from sympy import *
import sympy as sp 
from sympy import symbols, cos, diff
import mpl_toolkits
import random # Choosing a random element from a Python list with random.choice()
from sympy import var
from sympy import sympify
from sympy.plotting import plot3d
from numpy import linalg as LA
import time
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math as mp
from math import *
import math
import numpy
 
#Nombre des itérations maximal : 
global NBMAXITR
NBMAXITR=10000

#Déclaration des variables x et y : 
x,y=sp.symbols('x y',real=true)
v1 = var('x y')



In [3]:

#Les fonctions utilisées pour appliquer la méthode de gradient : 


#Convertir une chaine de caractéres en une expression mathématique : 
def expToFun(exp):
  
    exp=sympify(exp)           #Convertir une expression quelconque en un type Qu'on peut utiliser dans la bibliothéque SymPy.
    return lambdify([x,y],exp) #Convertir une expression SymPy en une fonction Python.

#Calculer le vecteur gradient (dérivé par rapport à x et par rapport y) : 
def calculVecteurgradiant(expr):
    expr=sympify(expr)
    d_p_x=sympify(sp.diff(expr,x))
    d_p_y=sympify(sp.diff(expr,y))
    d_p_x=lambdify([x,y],d_p_x)
    d_p_y=lambdify([x,y],d_p_y)
    return d_p_x,d_p_y

#Calculer la norme de vecteur gradient (la racine des élements du vecteur au carré): 
def calculNormeVecteurGradiant(ptX,ptY):
    return sqrt(ptX**2+ptY**2)


#Afficher un tableau comparatif entre les 3 méthodes (nb d'itérations, Temps d'éxecution et le minimum):
def cmpMethodes(expr):
    pas,eps,x0,y0=lecturePraam(expr)
    x1, y1,i1,eps1, A1, B1 , temps1=descentePasFix(expr,x0,y0,pas,eps,NBMAXITR)
    x2, y2,i2 ,eps2, A2, B2,  temps2,tabAlpha=CalculM_Gradian_pasVariable(expr,x0,y0,pas,eps)
    x3, y3,i3,eps3, A3, B3 , temps3=PasOptimal(expr,[x0,y0],eps)
    print("")
    print("f(x,y)=%s"%expr)
    print("")
    print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.")
    print("| methode de gradient | nb d'iterations  | temps d'execution \t\t | xmin \t\t     | y min  \t\t  |")
    print("----------------------------------------------------------------------------------------------------------------------------------")
    print("| MG pas fixe          |",i1,"         |",temps1,"     |",x1,"|",y1,"|")
    print("----------------------------------------------------------------------------------------------------------------------------------")
    print("| MG pas variable     |",i2,"         |",temps2,"      |",x2,"|",y2,"|")
    print("----------------------------------------------------------------------------------------------------------------------------------")
    print("| MG pas optimal      |",i3,"         |","Erreur","      |","Erreur","|","Erreur","|")
    print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.")


In [4]:
#Les fonctions de l'affichage : 

#Tracage de courbe en 3D : 
def tracageCourbe3D(expr):
    print("***** Courbe 3D de la fonction %s"%expr)
    a = np.linspace(-400, 400, 100)
    b = np.linspace(-400, 400, 100)
    X, Y = np.meshgrid(a, b)
    f=expToFun(expr)
    Z=f(X,Y)
    fig = plt.figure(figsize = (15,10))
    ax = plt.axes(projection='3d')
    ax.plot_surface(X, Y, Z, rstride=5, cstride=5,
        cmap='cool')
    ax.set_title("***** Courbe 3D de la fonction %s"%expr, fontsize = 13)
    ax.set_xlabel('x', fontsize = 11)
    ax.set_ylabel('y', fontsize = 11)
    ax.set_zlabel('Z', fontsize = 11)
    plt.show()
    
#Tracage des lignes de niveau :   
def tracageLigneNiveau(expr):
    xlist = np.linspace(-30.0, 30.0, 100)
    ylist = np.linspace(-30.0, 30.0, 100)
    X, Y = np.meshgrid(xlist, ylist)
    f=expToFun(expr)
    Z=f(X,Y)
    fig,ax=plt.subplots(1,1)
    cp = ax.contour(X, Y, Z)
    fig.colorbar(cp) # Add a colorbar to a plot
    ax.set_title('Lignes de niveaux du %s'%expr)
    ax.set_xlabel('x (cm)')
    ax.set_ylabel('y (cm)')
    plt.show()
    

In [5]:
#Lecture des paramétres : 

def lecturePraam(expr):
    pas,eps,x0,y0=0,0,0,0   # initialisation des paramétres à zero
    # vecteur par defaut 
    x0=1
    y0=1
    if(expr=="x*exp(-x**2-y**2)"):# vecteur par defaut pour f(x,y)=x*exp(-x**2-y**2)
        x0 = -0.5
        y0 = -0.5
    if(expr=="(1-x)**2+100*((y-(x**2))**2)"):# vecteur par defaut pour f(x,y)=x*exp(-x**2-y**2)
        x0 = 0.5
        y0 = 0.5  
    if(expr=="(x-2)**2+(y-3)**2"):
        x0 = 3
        y0 = 3
 
    print("")
    print("1) saisir vos propres valeurs de : eps / pas / vecteur X0 ? ")
    print("2) utiliser les valeurs par defaut eps=0.00001 / pas=0.1 / vecteur X0["+str(x0)+","+str(y0)+"] ? ")
    choix1 = int(input("choisir 1 / 2 : "))
    while (not (choix1 in range(1, 3))):
        choix1 = int(input("choisir 1 / 2 : "))
    
    #Lecture des paramétres saisis par l'utilisateur :
    if (choix1 == 1):

        #Lecture de EPS :
        print("Saisir la precision eps: \n")
        while True:    # lecture de eps avec un controle de saisie
            try:
                eps = np.float(input("eps doit etre sous la forme ..e.. (exemple 1e-6)"))
                break
            except ValueError:
                print("erreur de saisie .. veuillez ressaisir eps!!")  

        while (not (eps > 0)):
            while True:
                try:
                    eps = np.float(input("eps doit etre sous la forme ..e.. (exemple 1e-6)"))
                    break
                except ValueError:
                    print("erreur de saisie .. veuillez ressaisir eps!!")


        #Lecture de vecteur de départ :
        print("saisir X0 le vecteur de départ: ")
        print("X0=[x y]")

        # X :   lecture x avec un controle de saisie
        while True:
            try:
                x0 = float(input("saisir x : "))
                break
            except ValueError:
                print("erreur de saisie .. veuillez ressaisir x!!")
        
        # Y :    lecture y avec un controle de saisie
        while True:
            try:
                y0 = float(input("saisir y : "))
                break
            except ValueError:
                print("erreur de saisie .. veuillez ressaisir y!!")

        print(" le vecteur de départ X0=[", x0, " ", y0, "]")


        #Lecture du pas :
        print("saisir le pas(alpha) :\n ")
        while True:
            try:
                pas = np.double(input("pas(alpha) doit etre different de 0 .. saisir le pas : "))
                break

            except ValueError:
                print("erreur de saisie .. veuillez ressaisir le pas!!")
        while (not (pas > 0)):

            while True:
                try:
                    pas = np.double(input("le pas doit etre different de 0 .. saisir le pas : "))
                    break

                except ValueError:
                    print("erreur de saisie .. veuillez ressaisir le pas!!")


      #Les paramétres prédéfinis (dans la mémoire):
    if (choix1 == 2):
        pas = 0.1
        eps = 0.00001# 10-5
        
    return pas,eps,x0,y0


In [6]:
def affichageResultats(f1,A,B):    #cette fonction permet d'afficher tous les résultats correspondantes à chaque méthode 
    f=expToFun(f1)
    C = []     # le tableau C va prendre les f(x,y) ; avec x,y les cordonnées de chaque itérations 
    for j in range(len(A)):
        a = A[j]
        b = B[j]
        C.append(f(a, b))  # C=f(A,B)
    k = len(A) - 1 # k est le nombre final d'itérations
    print(len(A)," Itérations effectuées")
    print("les Coordonnées du minimum sont = [", A[k], ",", B[k], "]")
    #print(A)    
    #print(B)
    while true:
        print("veuillez:")
        print("1) Afficher les iterations")
        print("2) Afficher la courbe en 3D et les lignes de niveau")
        print("3) Le minimum : ")
        print("4) les trois  ")
        print("5) <- retour ")
        choix = int(input("choisir (1 / 2 / 3 / 4 / 5 ): "))
        while (not (choix in range(1, 6))):
            choix = int(input("choisir (1 / 2  / 3  / 4 / 5 ): "))
        
        if (choix == 1):
            for l in range(len(A)):  
                print("iteration -->", l, " : x=", A[l], "y=", B[l])   # affichage des itérations 
        if (choix == 2):# pour tracer le courbe en 3D et les itérations
          
            x = np.arange(-A[k] - 5, A[k] + 5, 0.2)   # x varie entre (-(xmin)-5) et ((xmin)+5) avec une pas de 0.2
            y = np.arange(-B[k] - 5, B[k] + 5, 0.2)   # y varie entre (-(ymin)-5) et ((ymin)+5) avec une pas de 0.2
            X, Y = np.meshgrid(x, y)      # X , Y sont le maillage 
            Z = f(X, Y)
            fig, ax = plt.subplots(1, 1)
            cp = ax.contourf(X, Y, Z)  # pour tracer les lignes de niveau
            fig.colorbar(cp)     # pour déterminer l'altitude 
            fig = plt.figure(figsize=(14, 9))
            ax = plt.axes(projection='3d')   # on va tracer les courbe en 3D
            ax.plot_surface(X, Y, Z, color="cyan")  # tracage du maillage en 3D
            ax.plot(A, B, C, color="r", linewidth=6)  # tracage des vecteurs des itérations 
            plt.show()
        if (choix == 3):   # le minimum
           
            print("les Coordonnées du minimum sont = [", A[k], ",", B[k], "]")
            print("")
        if (choix == 4):   # affichage de tous les résultats : courbe 3D , itérations , min 
            for l in range(len(A)):
                print("iteration -->", l, " : x=", A[l], "y=", B[l])
                print("")
            k = len(A) - 1
            print("les Coordonnées du minimum sont = [", A[k], ",", B[k], "]")
          
            x = np.arange(-A[k] - 5, A[k] + 5, 0.2)
            y = np.arange(-B[k] - 5, B[k] + 5, 0.2)
            X, Y = np.meshgrid(x, y)
            Z = f(X, Y)
            fig, ax = plt.subplots(1, 1)
            cp = ax.contourf(X, Y, Z)
            fig.colorbar(cp)  
            fig = plt.figure(figsize=(14, 9))
            ax = plt.axes(projection='3d')
            ax.plot_surface(X, Y, Z, color="cyan")
            ax.plot(A, B, C, color="r", linewidth=6)
            plt.show()
        if (choix == 5):
            break
    return

In [7]:
# Calcul du vecteur gradient & matrice hesseinne 

In [8]:
#La fonction partial retourne la dérivée partielle d'une fonction selon la variable passée en paramétre.
def partial(element, function):

    partial_diff = function.diff(element)
    return partial_diff

# La fonction gradient transforme une liste d'objets sympy en une matrice numpy.
def gradient(partials):
    
    grad = numpy.matrix([[partials[0]], [partials[1]]])
    return grad

#La fonction hessian Transforme une liste d'objets sympy en une matrice hessienne numpy.
def hessian(partials_second, cross_derivatives):
  
    hessianmat = numpy.matrix([[partials_second[0], cross_derivatives], [cross_derivatives, partials_second[1]]])
    return hessianmat

#La fonction grad() determine le vecteur gradient d'une fonction. et le retourne sous forme d'une matrice 
def grad():
    x, y = symbols('x y') 
    symbols_list = [x, y]
    partials, partials_second = [], []
 
    for element in symbols_list:
        partial_diff = partial(element, function) 
        partials.append(partial_diff)
    return gradient(partials)
    
#La fonction hess(exp) retourne la matrice Hessienne
def hess(exp):
    x, y = symbols('x y')
    symbols_list = [x, y]
    expr=sympify(exp)
    partials, partials_second = [], []#partials : les 1 ére derivé --- partials_second: les 2 nd derivé

    #calcule de derivé premier selon les symbole(x,y) est sauvegarde dans partials
    for element in symbols_list:
        partial_diff = partial(element, expr)
        partials.append(partial_diff)

    #-----------
    grad = gradient(partials)


    cross_derivatives = partial(symbols_list[0], partials[1])
    #----------- 
    # calcule de second derivée de derivés 1ere dans partials  % les differents symbole 
    for i in range(0, len(symbols_list)):
        partial_diff = partial(symbols_list[i], partials[i])
        partials_second.append(partial_diff)
     # calcule du la matrice hessienne    
    hessianmat = hessian(partials_second, cross_derivatives)
    return hessianmat


 

            

In [9]:
# Methode Gradiant pas variable: 

In [10]:
#Determiner le pas à chaque nouvelle itération : 
def calculNewAlpha(alpha):
    return alpha + 0.02*alpha # alpha suit une fonction liniaire croissante newAlpha=oldAlpha+ r*oldAlpha

#Méthode de gradient à pas variable : 
def CalculM_Gradian_pasVariable(expressionFonction,tempX,tempY,tempAlpha,eps): # return Xmin, Ymin, nbIteration, eps, tabX, tabY,temp,tabAlphaUtuliser,
    start = time.time()
    d_p_x,d_p_y =calculVecteurgradiant(expressionFonction)
    tabAlpha=[tempAlpha]
    tabX=[tempX]
    tabY=[tempY]
    i=0 # nombre maximal des iterations = NBMAXITR
    res=calculNormeVecteurGradiant(d_p_x(tempX,tempY),d_p_y(tempX,tempY))
    while(i<=NBMAXITR and res>eps):
        i+=1 #increment de l itirateur i
        x_grad=d_p_x(tempX,tempY) # calcul coordonnée x de vecteur gradiant
        y_grad=d_p_y(tempX,tempY) # calcul coordonnée y de vecteur gradiant
    # utilisant (-) car nous somme dans le cas de minimisation
    tempX = tempX - tempAlpha * x_grad # new point x
    tempY = tempY - tempAlpha * y_grad # new point y
    res=calculNormeVecteurGradiant(d_p_x(tempX,tempY),d_p_y(tempX,tempY))
    tempAlpha=calculNewAlpha(tempAlpha) #calculer le nouveau pas

    tabAlpha.append(tempAlpha) # sauvgarde les alphas utilisées dans le tableau tabAlpha 
    tabX.append(tempX) #tabX contient les x utilisée(parcoru) pour attendre le minimum 
    tabY.append(tempY)  #tabY contient les y utilisée(parcoru) pour attendre le minimum 
    
    end = time.time()
    temp=end-start
    return tempX,tempY,i,eps,tabX,tabY,temp,tabAlpha
def CalculM_Gradian_pasV(expr):
    tempAlpha,eps,tempX,tempY=lecturePraam(expr)#lecture des parametres du fonction 
    x,y,k,eps,tabX,tabY,temp,AlphaTab=CalculM_Gradian_pasVariable(expr,tempX,tempY,tempAlpha,eps)#affectation des resultats des calcules dans les differnts variable 
    affichageResultats(expr,tabX,tabY)# selon choix utilisateur les resultats sont afficher 
    return x,y,k,eps,tabX,tabY,temp,AlphaTab
  

In [11]:
# methode gradiant pas fix: 

In [12]:
def descentePasFix(expr, x1, y1, alpha, eps, maxIter):# return Xmin, Ymin, nbIteration, eps, tabX, tabY,temps
    A, B = [x1], [y1] # le tableau A va prendre les x et le tableau B va prendre les y de chaque itération 
    #A = []
    gradx, grady = calculVecteurgradiant(expr)  # gradx est le dérivée partielle de la fonction par rapport a x /// grady est le dérivée partielle de la fonction par rapport a y
    grad=calculNormeVecteurGradiant(gradx(x1,y1),grady(x1,y1)) # grad est le norme de gradient de f
    i = 0
    temps=0
    start = time.time() # pour calculer le temps d'execution on lance au début de boucle un chronométre 
    while abs(grad) > eps:  # tant que la norme du vecteur gradiant n'est pas approximativement nulle
        
        x1 = x1 - alpha * gradx(x1,y1)  # on effectue un petit pas vers le bas selon x
        y1 = y1 - alpha * grady(x1,y1)  # on effectue un petit pas vers le bas selon y
        grad=calculNormeVecteurGradiant(gradx(x1,y1), grady(x1,y1)) #grad va prendre le nouveau norme des nouveaux grad des x y  
        A.append(x1) #  A prend les x de chaque itération 
        B.append(y1) #  B prend les y de chaque itération
        
        i += 1  # on incremente le nombre d'iteration  
        if i > maxIter:  # si on dépasse le nombre maximale des itérations on va s'arréter
            end = time.time()
            temps=end-start
            return x1, y1,i,eps, A, B,temps
    end = time.time()
    temps=end - start # calcule temps ecouler pendant le calcule 
    #print(x1,y1,i,temps)
    return x1, y1,i,eps, A, B,temps


def methodegradientpasfixe(expr):
    pas,eps,x0,y0=lecturePraam(expr)  # on prend les paramétres pas , eps , x0 , y0
    x, y , i,eps,A, B , temps= descentePasFix(expr, x0, y0, pas, eps, NBMAXITR)
    affichageResultats(expr,A,B)  # affichage des résultats ( les itérations , le min ...)
    return x,y,A,B,i,temps

In [13]:
# Méthode gredient à pas optimal: 

In [14]:
#Definition de la fonction du gradient
def gradient_f(expr,X):
    expr=sympify(expr)
    d_p_x ,d_p_y=calculVecteurgradiant(expr)
    return([d_p_x(X[0],X[1]),d_p_y(X[0],X[1])])

#Definiton de la fonction a minimiser
def Omega (expr,X,alpha):
    gradx,grady=calculVecteurgradiant(expr)
    x1=X[0]-alpha*gradx(X[0],X[1])
    y1=X[1]-alpha*grady(X[0],X[1])
    return x1,y1

#Definiton de la fonction Rosenbrock (Pour le minimum global)
def Rosenbrock(X): 
    return ((X[0]-1)**2 + 10*(X[0]**2-X[1])**2)

def norme(u):
    x=u[0]
    y=u[1]
    return mp.sqrt(x**2+y**2)

#Recherche du pas optimal
def SectionDoree(expr,X0,a,b,tolerance):
    tau = (1+np.sqrt(5))/2
    it = 0
    err = b-a
    while np.abs(err)>tolerance :
        aprime = a + (b-a)/(tau*tau)
        bprime = a + (b-a)/tau
        c , d = Omega(expr,X0,aprime) , Omega(expr,X0,bprime)
        if c > d :
            a = aprime
        elif c < d :
            b = bprime
        else :
            a , b = aprime , bprime
        err = b -a
        it = it + 1
    return (a + b)/2

def PasOptimal (expr,X0,eps):# return Xmin, Ymin, nbIteration, eps, tabX, tabY,temps
    # X0 vecteur de depart X0=[x,y]
    alpha = SectionDoree(expr,X0,0,1,eps) #pas initial sur l'intervale [0 1]
    x=X0[0]
    y=X0[1]
    X = [x,y]
    listeX = [x]
    listeY = [y]
    k=0
    kmax=NBMAXITR
    gradx,grady=calculVecteurgradiant(expr) 
    temps=0#initialisation de temps
    start = time.time()
    while (norme([gradx(x,y),grady(x,y)])>eps) and (k<=kmax):
        X = [x,y]
        x,y=Omega(expr,X,alpha)
        k+=1
        listeX.append(x)
        listeY.append(y)
        X = [x,y]
        a=X0[0]-alpha*gradx(x,y)
        b= X0[1]-alpha*grady(x,y)
 
    end = time.time()
    temps=end-start
    return X[0],X[1],k,eps,listeX,listeY,temps

def methodegradientpasOptimal(expr):
    pas,eps,x0,y0=lecturePraam(expr)#lecture des parametres du fonction 
    x, y,i,eps, A, B , temps= PasOptimal(expr, [x0,y0], eps)
    affichageResultats(expr,A,B)# selon choix utilisateur les resultats sont afficher 
  

 

In [15]:
#fonction pour appeler menu1(projets1,2&4)
def myfunction(ch):
    global choix
    if(ch=='choisir'):
        choix=1
    elif (ch=='saisir'):
        choix=2
    else: 
        choix=3

In [16]:
#fonction pour récupérer la fonction choisie par l'utilisateur de la mémoire
def myfunction2(ch): 
    global choix1
    if(ch=='(1 − 𝑥)² + 100(𝑦 − 𝑥²)²'):
        choix1=1
    elif (ch=='xexp(-x²-y²)'):
        choix1 =2
    else: 
        choix1=3

In [17]:
#fonction pour récupérer la fonction saisie par l'utilisateur
def functionsaisie(ch): 
    global f
    f=ch

In [18]:

def inputNumber(prompt):
    while True :
        try:
            num = float(input(prompt))
            break
        except ValueError:
            pass
    return num

def inputChar(prompt):
    while True :
        try:
            c = str(input(prompt))
            break
        except ValueError:
            print("Valeur incorrecte !")
            pass
    return c

    #menu with numbers as a choice
def displayMenu(opt) :
    for i in range(len(opt)):
        print("{:d}. {:s}".format(i+1, opt[i]))
    choice = 0
    while not(np.any(choice == np.arange(len(opt))+1)):
        choice = inputNumber ("Entrez votre choix: ")
    return choice
    
    #menu with Caracters as a choice
def displayMenuFunction(opt) :
    for i in range(len(opt)):
        print("{:s}".format(opt[i]))

    choice = inputChar ("Entrez votre choix: ")
    return choice

def choixGradiant(fct):
    print("")
    print("Appliquant la methode de gradiant avec la fonction %s "%fct )
    menuItemsFunction = np.array(["a.appliquer la méthode à pas fixe.","b.à pas variable.","c.à pas optimale.","d.Comparatif entre les 3 méthodes."])
    choice=displayMenuFunction(menuItemsFunction)
    if choice.lower()=="a":
        methodegradientpasfixe(fct)
    elif choice.lower()=="b":
        CalculM_Gradian_pasV(fct)
    elif choice.lower()=="c":
   # methodegradientpasOptimal(fct)       #ne fonctionne pas
        print("Erreur: il y'a un problème dans le code.") #projet final: on a modifié l'affichage par ce message
    elif choice.lower()=="d":
        cmpMethodes(fct)

# read user function 
def readFunction():
####################cette partie est exécutée dans l'interface####################
#       print("""
#   \n Aide :  \n 
#   Tapez x pour la variable (exemple f(x) = 2*x+ x/2 ) \n 
#   Tapez a**b pour 'a' puissance 'b' (exemple : 2 puissance 4 => 2**4 ) \n
#   Pour les fonctions cos et sin  : f(x) = cos(2*x) || f(x) = sin(4*x) \n
#   Pour la fonction exponentielle tapez 'exp' : exemple f(x) = 2*exp(x) \n
#   Pour la fonction Logarithme tapez 'ln(x)' : exemple f(x) = ln(2*x) \n
#   \n 
#   **************************************************************** \n
#   exemple d'une fonction  \n
#   f(x) = e(5x)/2x+4  ======> le syntaxe à écrire:======> exp(5*x)/2*x+4
 
#   """)
 
#       while True:
#         try:
# #             print("Donnez une fonction de votre choix\n")

#             inputFunction = input("\n f(x) = ") ##########à modifierrrrr
            inputFunction = f
            expr = sympify(inputFunction)# Changement d'une chaine a une fonction mathématique 
            globalUsedFunction=expToFun(expr)
            menu2(inputFunction) 
####################cette partie est exécutée dans l'interface####################
#         except:
#             print("!!!! Verifiez le syntaxe de votre fonction !!!!")
#             print("""
#             Utiliser x pour la varibale. exemple : f(x)= 2*x+5*x+3 \n
#             -----------------------------------------------\n
#             |        f(x)         | syntaxe à écrire      |\n
#             -----------------------------------------------
#             |     ax              |   a*x                 |\n
#             |     (ax+b)/(cx+d)   |  (a*x+b)/(c*x+d)      |\n
#             |     x²              |   x**2                |\n
#             |     x power a       |   x**a                |\n
#             |     log(x)          |   ln(x)               |\n
#             |     cos(x)          |   cos(x)              |\n
#             |     sin(x)          |   sin(x)              |\n
#             |     cos(ax)         |   cos(a*x)            |\n
#             |     sin(ax)         |   sin(a*x)            |\n
#             |     e(x)            |   exp(x)              |\n
#             -----------------------------------------------\n
        
#             """)
#             continue
#         else:
#             break

def menu2(expr):
    print("")
    print("a. Tracer la courbe de %s "%expr)
    print("b. Tracer les lignes de niveaux")
    print("c. Calculer le vecteur gradient de la fonction %s"%expr) 
    print("d. Calculer la matrice Hessienne") 
    print("e. Appliquer la méthode de gradient") 
    print("f. <- Revenir au menu principal") 
    print("g. <- Revenir a l interface") 
    choix = input("Entrez votre choix (a/b/c/d/e/f): ")

    if choix.lower() == 'a':
        tracageCourbe3D(expr)
        menu2(expr)
    elif choix.lower() == 'b':
        print("lignes de niveaux")
        tracageLigneNiveau(expr)
        menu2(expr)
    elif choix.lower() == 'c':
       
        p_grad_x,p_grad_y=calculVecteurgradiant(expr)
     
        p_grad_x=sympify(sp.diff(expr,x))
        p_grad_y=sympify(sp.diff(expr,y))
        print("\nle vecteur gradiant de la fonction %s est:"%expr)
        print("< ",p_grad_x,";",p_grad_y," >")
        menu2(expr)
    elif choix.lower() == 'd':
        print("la matrice Hessienne de la fonction f(x,y)=%s"%expr)
        print("H=",hess(expr))
        menu2(expr)  
    elif choix.lower() == 'e':
        choixGradiant(expr)
        menu2(expr)
    elif choix.lower() == 'f':
        
        menu() 
    elif choix.lower() == 'g':
        
        sys.exit('byee') #pour migrer du console vers l'interface
    else:
        print("!!!!!!!!!!Vérifiez votre choix SVP!!!!!!!!")
        while True:
            menu2(expr)
            break
def menu():
####################cette partie est exécutée dans l'interface####################
#     print("** Menu Principal **")
#     menuItems = np.array([
#         "Choisir une fonction de la mémoire.",
#         "Enter votre propre fonction.",
#         "Quitter.\n"])
#     choix = displayMenu(menuItems);
    
    if choix == 1:
####################cette partie est exécutée dans l'interface####################
#         print("Choisir une fonction de la mémoire")
#         menuItems = np.array([
#         "1- f1(x,y)= (1-x)**2+100*((y-(x**2))**2)",
#         "2- f2(x,y)= x*exp(-x**2-y**2)",
#         "3- 𝑓(𝑥,𝑦) =1/2〈𝐴𝑥, 𝑥〉 − 〈𝑏, 𝑥〉; 𝐴=[[3,-1,0,0,0],[-1,12,-1,0,0],[0,-1,24,-1,0],[0,0,-1,48,-1],[0,0,0,-1,96]]; 𝑏=[1,2,3,4,5]",
#         ]) 
#         choix1 = displayMenu(menuItems);
        if choix1 == 1:
            print("\n\nPrener en considération que le choix e ne fonctionne pas pour cette fonction")
            menu2("(1-x)**2+100*((y-(x**2))**2)")
            
                          
        elif choix1 == 2: 
            menu2("x*exp(-x**2-y**2)") 
        elif choix1 == 3: 
            print("Cette fonction est irréalisable avec ce projet !")
        #menu2("𝑓(𝑥,𝑦) =1/2〈𝐴𝑥, 𝑥〉 − 〈𝑏, 𝑥〉")
       
    elif choix == 2:
        #print("Donner votre propre fonction")
        readFunction()#lecture des fonctions saisie par l'utilisateur
    elif choix == 3:
        print ('Merci d\'avoir utilisé notre programme')
        return # out of the programme
    else:
        print("!!!!!!!!!!Vérifiez votre choix svpp!!!!!!!!")
#         while True:
#             menu ()
#             break


In [19]:
#--------------MAIN------------------
print("Bienvenue:")
print("PROJET 1:")
print("Notre programme permet de tracer une courbe en 3D, les lignes de niveaux, calculer le vecteur gradient et calculer la matrice Hessienne\n ainsi que l’implémentation des méthodes de gradient à pas fixe, variable et optimal d\'une fonction dans R2")

#menu()

 

Bienvenue:
PROJET 1:
Notre programme permet de tracer une courbe en 3D, les lignes de niveaux, calculer le vecteur gradient et calculer la matrice Hessienne
 ainsi que l’implémentation des méthodes de gradient à pas fixe, variable et optimal d'une fonction dans R2
